In [ ]:
# !pip install pydub

In [ ]:
import os
import sys
import numpy as np
import pandas as pd

from tqdm import tqdm
#from pydub import AudioSegment
from IPython.display import Audio as AudioI
#from sklearn.model_selection import train_test_split

curr_parh = os.getcwd()
sys.path.append('../src')
from config import DEFAULT_BUCKET, DEFAULT_REGION  
from gdsc_utils import upload_to_s3, PROJECT_DIR

In [ ]:
os.chdir(f'{curr_parh}/..')
os.getcwd()

In [ ]:
threshold = 7
path_trn_chunked = 'data/data_chunks_7sec_small/train_chunked/'
path_val_chunked = 'data/data_chunks_7sec_small/val_chunked/'
path_tst_chunked = 'data/data_chunks_7sec_small/test_chunked/'

In [ ]:
def chunks(path_original, path_chunked, threshold, random_frac=None):
    
    mtd = pd.read_csv(path_original+'metadata.csv')             
    label_flag = False
    if 'label' in list(mtd.columns):
        label_flag = True
        
    if (random_frac is not None) & (label_flag):        
        _, file_tst = train_test_split(mtd.file_name, test_size=random_frac, stratify=mtd.label, random_state=42)
        mtd = mtd[mtd.file_name.isin(file_tst.values)].reset_index(drop=True)
    if (random_frac is not None) & (not label_flag):        
        _, file_tst = train_test_split(mtd.file_name, test_size=random_frac, random_state=42)
        mtd = mtd[mtd.file_name.isin(file_tst.values)].reset_index(drop=True)      
        
    if label_flag:    
        new_metadata_chunked = pd.DataFrame(columns=['file_name', 'label'])
    else:
        new_metadata_chunked = pd.DataFrame(columns=['file_name'])
    
    
    for i in tqdm(range(len(mtd))):
        file_name = mtd.iloc[[i]]['file_name'].values[0]
        if label_flag:
            label = mtd.iloc[[i]]['label'].values[0]
        audio = AudioSegment.from_wav(path_original+file_name)
        length = len(audio)/1000
        sec = threshold * 1000
        if length > threshold:
            n = int(length//threshold)
            for j in range(0, n+1):
                new_audio = audio[sec*j:sec*(j+1)]
                new_filename = f'{file_name[:-4]}'+'_cut'+str(j)+'.wav'
                new_path = f'{path_chunked+new_filename}'
                new_len = len(new_audio) / 1000 
                if new_len >= 1:
                    new_audio.export(path_chunked+new_filename, format="wav")
                    if label_flag:
                        new_metadata_chunked.loc[len(new_metadata_chunked)] = [new_filename, label]
                    else:
                        new_metadata_chunked.loc[len(new_metadata_chunked)] = [new_filename]                 
        else:
            audio.export(path_chunked + file_name, format="wav")
            if label_flag:
                new_metadata_chunked.loc[len(new_metadata_chunked)] = [file_name, label]
            else:
                new_metadata_chunked.loc[len(new_metadata_chunked)] = [new_filename]            
    return new_metadata_chunked

In [ ]:
new_metadata_train_chunked = chunks(path_original='data/train/', path_chunked=path_trn_chunked, threshold=threshold, random_frac=0.3)
new_metadata_train_chunked.to_csv(path_trn_chunked + 'metadata.csv', index=False)

In [ ]:
new_metadata_val_chunked = chunks(path_original='data/val/', path_chunked=path_val_chunked, threshold=threshold)
new_metadata_val_chunked.to_csv(path_val_chunked + 'metadata.csv', index=False)

In [ ]:
new_metadata_tst_chunked = chunks(path_original='data/test/', path_chunked=path_tst_chunked, threshold=threshold)
new_metadata_tst_chunked.to_csv(path_tst_chunked + 'metadata.csv', index=False)

In [ ]:
new_metadata_train_chunked.shape, new_metadata_val_chunked.shape, new_metadata_tst_chunked.shape

# Upload to S3 bucket

In [ ]:
path_local = '/root/data/data/data_chunks_7sec_small'
path_s3 = 'data_chunks_7sec_small'

In [ ]:
for path, directories, files in os.walk(path_local+'/train_chunked'):
    for file in tqdm(files):
        if file != 'metadata-checkpoint.csv':
            upload_to_s3(local_path=f'{path_local}/train_chunked/{file}',
                         s3_path=f'{path_s3}/train/{file}',
                         bucket='sagemaker-us-east-1-292159885427')

In [ ]:
for path, directories, files in os.walk('/root/data/data/val'):
    for file in tqdm(files):
        if file != 'metadata-checkpoint.csv':
            upload_to_s3(local_path=f'/root/data/data/val/{file}',
                         s3_path=f'{path_s3}/val/{file}',
                         bucket='sagemaker-us-east-1-292159885427')

In [ ]:
for path, directories, files in os.walk('/root/data/data/test'):
    for file in tqdm(files):
        if file != 'metadata-checkpoint.csv':
            upload_to_s3(local_path=f'/root/data/data/test/{file}',
                         s3_path=f'{path_s3}/test/{file}',
                         bucket='sagemaker-us-east-1-292159885427')

In [ ]:
upload_to_s3(local_path=f'/root/data/data/labels-BACKUP.json',
             s3_path=f'{path_s3}/labels.json',
             bucket='sagemaker-us-east-1-292159885427')